In [2]:
! wget --header="Host: doc-14-24-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/drive/folders/1eY4pTqtGtXSAYVzaHwWAK2Cq4bHb1Zkq" --header="Cookie: AUTH_t5qon8tobs4crgo9e5mfjphsf8l1st7h_nonce=ejl9auk692ojc" --header="Connection: keep-alive" "https://doc-14-24-docs.googleusercontent.com/docs/securesc/nji0agiogjqo3r2b8t0lib5qttb9sgab/1g4tsv0nhpbnh00lf103vbnrsli7gbt0/1587878250000/00484516897554883881/00113621618736444193/1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu?e=download&authuser=0&nonce=ejl9auk692ojc&user=00113621618736444193&hash=nqsb654ga5o3u03g898m52q5r8upk3f7" -c -O 'rvl-cdip.rar'

--2020-04-26 05:18:58--  https://doc-14-24-docs.googleusercontent.com/docs/securesc/nji0agiogjqo3r2b8t0lib5qttb9sgab/1g4tsv0nhpbnh00lf103vbnrsli7gbt0/1587878250000/00484516897554883881/00113621618736444193/1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu?e=download&authuser=0&nonce=ejl9auk692ojc&user=00113621618736444193&hash=nqsb654ga5o3u03g898m52q5r8upk3f7
Resolving doc-14-24-docs.googleusercontent.com (doc-14-24-docs.googleusercontent.com)... 172.217.7.129, 2607:f8b0:4004:805::2001
Connecting to doc-14-24-docs.googleusercontent.com (doc-14-24-docs.googleusercontent.com)|172.217.7.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/rar]
Saving to: ‘rvl-cdip.rar’

rvl-cdip.rar            [             <=>    ]   4.34G  66.4MB/s    in 88s     

2020-04-26 05:20:26 (50.8 MB/s) - ‘rvl-cdip.rar’ saved [4660541790]



In [3]:
import os 
print(os.getcwd())
print(os.listdir())

/content
['.config', 'rvl-cdip.rar', 'sample_data']


In [4]:
! pip install pyunpack

  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5163 sha256=1308b703c90888b7621d4954bfd5010336effe0f50061ed5bf3572b271d4e791
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [5]:
!pip install patool

     |████████████████████████████████| 81kB 8.3MB/s 


In [0]:
from pyunpack import Archive
Archive('rvl-cdip.rar').extractall('image_data')

In [0]:
import pandas as pd
datafile = pd.read_csv('/content/image_data/labels_final.csv',dtype=str)

In [0]:
import tensorflow as tf
ImageFlow = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.25)

In [9]:
datafile.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [0]:
datafile['label'].unique()

array(['3', '2', '7', '4', '1', '13', '5', '6', '0', '15', '11', '8',
       '14', '10', '12', '9'], dtype=object)

In [10]:
train_generator = ImageFlow.flow_from_dataframe(
        dataframe=datafile,
        directory='/content/image_data/data_final/',
        x_col="path",
        y_col="label",
        target_size=(156, 156),
        shuffle = True,
        subset = "training",
        batch_size=128,
        class_mode='categorical')

Found 36000 validated image filenames belonging to 16 classes.


In [11]:
val_generator = ImageFlow.flow_from_dataframe(
        dataframe=datafile,
        directory='/content/image_data/data_final/',
        x_col="path",
        y_col="label",
        target_size=(156, 156),
        shuffle = True,
        subset = "validation",
        batch_size=128,
        class_mode='categorical')

Found 12000 validated image filenames belonging to 16 classes.


Import files

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
#from keras.layers import Dropout, Flatten, Dense , Conv1D , Conv2D , Dense  , MaxPooling2D
from tensorflow.keras import applications
from tensorflow.keras.callbacks import EarlyStopping
import datetime

Model1

In [14]:
# Clear any logs from previous runs
!rm -rf ./logs/
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)


In [15]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet',input_shape=(156,156,3))
x = vgg.output

58892288/58889256 [==============================] - 0s 0us/step


In [0]:
for layer in vgg.layers:
  layer.trainable = False

In [17]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 156, 156, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 156, 156, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 156, 156, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 78, 78, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 78, 78, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 78, 78, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 39, 39, 128)       0     

In [0]:
l1 = tf.keras.layers.Conv2D(64,  kernel_size=(3,3), strides=1, padding='same', data_format='channels_last', dilation_rate=1, activation='relu')(x)
l2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=1, padding='same', data_format=None)(l1)
flatten = tf.keras.layers.Flatten()(l2)
l3 = tf.keras.layers.Dense(50,activation='relu')(flatten)
l4 = tf.keras.layers.Dense(32,activation='relu')(l3)
output = tf.keras.layers.Dense(16,activation='softmax')(l4)

In [0]:
model = tf.keras.Model(inputs = vgg.input, outputs = output)
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [0]:
tf.keras.backend.clear_session()

In [21]:
model.fit_generator(
    train_generator, steps_per_epoch = 100, validation_data = val_generator, validation_steps = 64,  epochs=20 , callbacks=tensorboard_callback
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
100/100 [==============================] - 113s 1s/step - loss: 3.2323 - accuracy: 0.1494 - val_loss: 2.5370 - val_accuracy: 0.2065
Epoch 2/20
100/100 [==============================] - 102s 1s/step - loss: 2.4456 - accuracy: 0.2366 - val_loss: 2.3473 - val_accuracy: 0.2602
Epoch 3/20
100/100 [==============================] - 99s 993ms/step - loss: 2.2989 - accuracy: 0.2787 - val_loss: 2.2491 - val_accuracy: 0.2804
Epoch 4/20
100/100 [==============================] - 86s 855ms/step - loss: 2.1797 - accuracy: 0.3218 - val_loss: 2.1503 - val_accuracy: 0.3384
Epoch 5/20
100/100 [==============================] - 94s 942ms/step - loss: 2.1122 - accuracy: 0.3451 - val_loss: 2.0805 - val_accuracy: 0.3588
Epoch 6/20
100/100 [==============================] - 95s 950ms/step - loss: 2.0450 - accuracy: 0.3665 - val_loss: 2.0356 - val_accuracy: 0.3689
Epoch 7/20
100/100 [==============================] - 80s 

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Conclusion : We can conclude that we can get around 0.5 accuracy both train as validation for this Model

Model2

In [23]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

In [0]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet',input_shape=(156,156,3))
x = vgg.output
for layer in vgg.layers:
  layer.trainable = False

l1 = tf.keras.layers.Conv2D(64,kernel_size=(4,4), strides=(1,1), padding='same', data_format='channels_last', dilation_rate=1, activation='relu')(x)
l2 = tf.keras.layers.Conv2D(40,kernel_size=(4,4),activation='relu',strides=(1,1), padding='same')(l1)

l3 = tf.keras.layers.Flatten()(l2)
output = tf.keras.layers.Dense(16,activation='softmax')(l3)

In [0]:
model = tf.keras.Model(inputs = vgg.input, outputs = output)
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [30]:
model.fit_generator(
    train_generator, steps_per_epoch = 100, validation_data = val_generator, validation_steps = 64,  epochs=20 , callbacks=tensorboard_callback
)

Epoch 1/20
100/100 [==============================] - 78s 776ms/step - loss: 2.7286 - accuracy: 0.2819 - val_loss: 2.0122 - val_accuracy: 0.3982
Epoch 2/20
100/100 [==============================] - 78s 780ms/step - loss: 1.9414 - accuracy: 0.4208 - val_loss: 1.8345 - val_accuracy: 0.4549
Epoch 3/20
100/100 [==============================] - 77s 767ms/step - loss: 1.7671 - accuracy: 0.4678 - val_loss: 1.7458 - val_accuracy: 0.4812
Epoch 4/20
100/100 [==============================] - 76s 757ms/step - loss: 1.6586 - accuracy: 0.4995 - val_loss: 1.6881 - val_accuracy: 0.4966
Epoch 5/20
100/100 [==============================] - 76s 764ms/step - loss: 1.6212 - accuracy: 0.5116 - val_loss: 1.6156 - val_accuracy: 0.5091
Epoch 6/20
100/100 [==============================] - 76s 760ms/step - loss: 1.5435 - accuracy: 0.5258 - val_loss: 1.6042 - val_accuracy: 0.5248
Epoch 7/20
100/100 [==============================] - 76s 758ms/step - loss: 1.5007 - accuracy: 0.5416 - val_loss: 1.5799 - val_ac

In [0]:
%tensorboard --logdir logs/fit

Conclusion : We can conclude that we can get around 0.6 accuracy train as well as validation for this model

Model3

In [27]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

In [0]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet',input_shape=(156,156,3))
x = vgg.output

for layer in vgg.layers:
  layer.trainable = False
  
l1 = tf.keras.layers.Conv2D(64,kernel_size=(4,4), strides=(1,1), padding='same', data_format='channels_last', dilation_rate=1, activation='relu')(x)

l2 = tf.keras.layers.Conv2D(40,kernel_size=(4,4),activation='relu',strides=(1,1), padding='same')(l1)
l3 = tf.keras.layers.Flatten()(l2)
output = tf.keras.layers.Dense(16,activation='softmax')(l3)

In [0]:
model = tf.keras.Model(inputs = vgg.input, outputs = output)
for layer in vgg.layers[:-6]:
  layer.trainable = False
model.compile(loss='categorical_crossentropy',optimizer = tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [45]:
model.fit_generator(
    train_generator, steps_per_epoch = 100, validation_data = val_generator, validation_steps = 64,  epochs=20 , callbacks=tensorboard_callback
)

Epoch 1/20
100/100 [==============================] - 76s 759ms/step - loss: 2.7271 - accuracy: 0.2923 - val_loss: 2.0481 - val_accuracy: 0.3955
Epoch 2/20
100/100 [==============================] - 79s 790ms/step - loss: 1.9287 - accuracy: 0.4277 - val_loss: 1.8288 - val_accuracy: 0.4597
Epoch 3/20
100/100 [==============================] - 77s 775ms/step - loss: 1.7765 - accuracy: 0.4770 - val_loss: 1.7400 - val_accuracy: 0.4816
Epoch 4/20
100/100 [==============================] - 77s 768ms/step - loss: 1.6496 - accuracy: 0.5081 - val_loss: 1.6559 - val_accuracy: 0.5034
Epoch 5/20
100/100 [==============================] - 78s 782ms/step - loss: 1.5956 - accuracy: 0.5173 - val_loss: 1.6279 - val_accuracy: 0.5126
Epoch 6/20
100/100 [==============================] - 81s 810ms/step - loss: 1.5274 - accuracy: 0.5344 - val_loss: 1.5748 - val_accuracy: 0.5327
Epoch 7/20
100/100 [==============================] - 81s 807ms/step - loss: 1.4872 - accuracy: 0.5474 - val_loss: 1.5646 - val_ac

In [0]:
%tensorboard --logdir logs/fit

Conclusion : we can conclude that we can get 0.59 as validation and 0.64 as train accuracy for this model

Finally we can conclude that 3rd gives best accuracy from all models.